In [1]:
!pip install opencv-python google-cloud-vision


  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp311-cp311-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
    --------------------------------------- 0.8/39.5 MB 2.1 MB/s eta 0:00:19
   - -------------------------------------- 1.3/39.5 MB 2.3 MB/s eta 0:00:17
   - -------------------------------------- 1.8/39.5 MB 2.5 MB/s eta 0:00:15
   -- ------------------------------------- 2.4/39.5 MB 2.5 MB/s eta 0:00:15
   -- ------------------------------------- 2.9/39.5 MB 2.4 MB/s eta 0:00:16
   --- ------------------------------------ 3.4/39.5 MB 2.4 MB/s eta 0:00:15
   --- ------------------------------------ 3.9/39.5 MB 2.4 MB/s eta 0:00:15
   ---- ----------------------------------- 4.5/39.5 MB 2.5 MB/s eta 0:00:15
   ---- ----------------------------

## Para Linux

In [ ]:
!export GOOGLE_APPLICATION_CREDENTIALS="nimble-root-457808-r2-b639a6729402.json"

'export' is not recognized as an internal or external command,
operable program or batch file.


## Para Windows

In [17]:
%env GOOGLE_APPLICATION_CREDENTIALS="C:\Users\User\Desktop\Master\TFM\test reconocimiento de imagen\nimble-root-457808-r2-b639a6729402.json"

env: GOOGLE_APPLICATION_CREDENTIALS="C:\Users\User\Desktop\Master\TFM\test reconocimiento de imagen\nimble-root-457808-r2-b639a6729402.json"


In [14]:
import cv2
import io
from google.cloud import vision

In [21]:
# detect_people_webcam.py
# Uso: Captura vídeo de la webcam y detecta personas con Google Cloud Vision API.
#
# Requisitos:
#  - Python 3.x
#  - Biblioteca OpenCV y cliente de Vision API:
#      pip install opencv-python google-cloud-vision
#
# Configuración de credenciales de GCP:
#  * En Linux/macOS (bash/zsh):
#      export GOOGLE_APPLICATION_CREDENTIALS="/ruta/a/tu/credencial.json"
#  * En Windows (PowerShell):
#      $env:GOOGLE_APPLICATION_CREDENTIALS = 'C:\ruta\a\tu\credencial.json'
#    o en el Símbolo del sistema (CMD):
#      set GOOGLE_APPLICATION_CREDENTIALS=C:\ruta\a\tu\credencial.json
#  * En Jupyter Notebook:
#      %env GOOGLE_APPLICATION_CREDENTIALS=C:\ruta\a\tu\credencial.json
#
# Opcional: definir la variable de entorno dentro del script (descomenta y ajusta la ruta):
import os
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\User\Desktop\Master\TFM\test reconocimiento de imagen\nimble-root-457808-r2-b639a6729402.json'
#
# Asegúrate de habilitar la API Vision y de que la cuenta de servicio tenga permisos de "Cloud Vision API User".

# Si no has configurado la variable externamente, puedes usar la línea siguiente:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\User\Desktop\Master\TFM\test reconocimiento de imagen\nimble-root-457808-r2-b639a6729402.json'

def main():
    # Inicializa el cliente de Vision API
    client = vision.ImageAnnotatorClient()

    # Abre la webcam (índice 0)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: No se pudo abrir la webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Codifica el frame como JPEG en memoria
        ret2, buf = cv2.imencode('.jpg', frame)
        if not ret2:
            continue
        content = buf.tobytes()

        # Prepara la imagen para enviar a Vision API
        image = vision.Image(content=content)

        # Detección de objetos
        response = client.object_localization(image=image)
        objects = response.localized_object_annotations

        # Dimensiones del frame
        h, w, _ = frame.shape

        # Dibuja bounding boxes para cada persona
        for obj in objects:
            if obj.name.lower() == 'person':
                verts = obj.bounding_poly.normalized_vertices
                pts = [(int(v.x * w), int(v.y * h)) for v in verts]
                x_coords = [p[0] for p in pts]
                y_coords = [p[1] for p in pts]
                x_min, x_max = min(x_coords), max(x_coords)
                y_min, y_max = min(y_coords), max(y_coords)
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                cv2.putText(frame, 'Person', (x_min, y_min - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Muestra el resultado
        cv2.imshow('Detección de Personas', frame)

        # Salir con 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



In [ ]:
if __name__ == '__main__':
    main()

KeyboardInterrupt: 

: 